In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from tqdm.notebook import tqdm
import scipy.fftpack as fft
import cv2

import pydicom
import datetime
from pydicom.dataset import Dataset, FileDataset, FileMetaDataset
from pydicom.filewriter import correct_ambiguous_vr

pi = np.pi

In [ ]:
def getSource(alfa, r):
    Xe = r * np.cos(alfa)
    Ye = r * np.sin(alfa)
    
    return (Xe, Ye)

#n - ilosc detektorow, alfa - przesuniecie wszystkeigo, phi - rozstawienie detektorow, r - promien kola
def getDetectors(n, alfa, phi, r):
    detectors = []
    
    # D0
    Xd = r * np.cos(alfa + pi - (phi/2))
    Yd = r * np.sin(alfa + pi - (phi/2))

    detectors.append( (Xd, Yd) )
    
    # D1 ... Dn-1
    for i in range(1,n):
        Xd = r * np.cos(alfa + pi - (phi/2) + (i * (phi/(n-1))))
        Yd = r * np.sin(alfa + pi - (phi/2) + (i * (phi/(n-1))))
        
        detectors.append( (Xd, Yd) )
    
    return detectors

def transformToPictureCoordinates(point, picHeight, picWidth):
    X = point[0] + (0.5*picWidth)
    Y = point[1] + (0.5*picHeight)
    
    X = int(np.floor(X))
    Y = int(np.floor(Y))
    return ( X, Y )

In [ ]:
def bresenhamL(x1, y1, x2, y2):
    points = []
    dx = x2 - x1
    dy = y2 - y1
    m = 1
    if dy < 0:
        m = -1
        dy = -dy
    j = y1
    e = (2 * dy) - dx
    
    for i in range(x1, x2):
        points.append((i, j))
        if e > 0:
            j += m
            e += 2 * (dy - dx)
        else:
            e += 2 * dy
    return points

def bresenhamH(x1, y1, x2, y2):
    points = []
    dx = x2 - x1
    dy = y2 - y1
    m = 1
    if dx < 0:
        m = -1
        dx = -dx
    i = x1
    e = (2 * dx) - dy
    
    for j in range(y1, y2):
        points.append((i, j))
        if e > 0:
            i += m
            e += 2 * (dx - dy)
        else:
            e += 2 * dx
    return points

def bresenham(x1, y1, x2, y2):
    if abs(y2 - y1) < abs(x2 - x1):
        if x1 > x2:
            return bresenhamL(x2, y2, x1, y1)
        else:
            return bresenhamL(x1, y1, x2, y2)
    else:
        if y1 > y2:
            return bresenhamH(x2, y2, x1, y1)
        else:
            return bresenhamH(x1, y1, x2, y2)

In [ ]:
def absorption(img, x1, y1, x2, y2):
    height = img.shape[0]
    width = img.shape[1]
    
    s = 0
    
    for (x, y) in bresenham(x1, y1, x2, y2):
        if x >= 0 and x < width and y >= 0 and y < height:
            s += img[y][x]
    return s

In [ ]:
def rmse(original, estimated):
    return np.sqrt(np.mean( (estimated-original) ** 2))

In [ ]:
def img2sinogram(img, alfaDelta, n, phi):
    alfaDelta = np.radians(alfaDelta)
    phi = np.radians(phi)
    
    picHeight = img.shape[0]
    picWidth = img.shape[1]
    d = np.sqrt( picWidth**2 + picHeight**2 )
    r =  d / 2
    
    lines = []
    alfa = 0.00000001
    pbar = tqdm(total=360, unit='°')
    while alfa < 2 * pi:
        
        source = getSource(alfa, r)
        pSource = transformToPictureCoordinates(source, picWidth, picHeight)
        
        detectors = getDetectors(n, alfa, phi, r)
        pDetectors = [transformToPictureCoordinates(x, picWidth, picHeight) for x in detectors]
        
        line = []
        
        for detector in pDetectors:
            a = absorption(img, pSource[0], pSource[1], detector[0], detector[1])
            line.append(a)
            
        
        lines.append(line)
        alfa += alfaDelta
        pbar.update(np.degrees(alfaDelta))
    
    pbar.close()
    lines = np.stack(lines, axis=0)    
    
    normalized = (lines - lines.min()) / (lines.max() - lines.min())
    
    return normalized

In [ ]:
def sinogram2img(sin, picHeight, picWidth, phi, iteration, filtered, show=False, original=None):
    sinHeight = sin.shape[0]
    sinWidth = sin.shape[1]
    phi = np.radians(phi)
    d = np.sqrt( picWidth**2 + picHeight**2 )
    r =  d / 2
    
    alfaDelta = np.radians(360 / sinHeight)
    n = sinWidth
    
    img = np.zeros((picHeight, picWidth))
    
    if filtered:
        f = fft.rfft(sin, axis=1)
        ramp = np.floor(np.arange(0.5, f.shape[1]//2 + 0.1, 0.5))
        sin = fft.irfft(f * ramp, axis=1)
    
    alfa = 0
    line = 0
    pbar = tqdm(total=360, unit='°')
    while alfa < 2 * pi and line < iteration:
        
        source = getSource(alfa, r)
        pSource = transformToPictureCoordinates(source, picWidth, picHeight)
        
        detectors = getDetectors(n, alfa, phi, r)
        pDetectors = [transformToPictureCoordinates(x, picWidth, picHeight) for x in detectors]
        
        for row, detector in enumerate(pDetectors):
            for (x, y) in bresenham(pSource[0], pSource[1], detector[0], detector[1]):
                if x >= 0 and x < picWidth and y >= 0 and y < picHeight:
                    img[y][x] += sin[line][row]
            
        
        alfa += alfaDelta
        line += 1
        pbar.update(np.degrees(alfaDelta))
        
    pbar.close()
    
    normalized = (img - img.min()) / (img.max() - img.min())
    error = None
    
    if filtered:
        normalized_uint8 = np.uint8(normalized * 255)
        equalized_uint8 = cv2.equalizeHist(normalized_uint8)
        normalized = np.float64(equalized_uint8) / 255.0
        
    if original is not None:
        error = rmse(original, normalized)
        if show:
            print("RMSE = ", error)
    
    if show:
        plt.figure(figsize = (10, 10))
        plt.imshow(normalized, cmap='binary_r')
    else:
        return (np.array(normalized), error)

In [ ]:
def interactive_sinogram(sinogram):
    max_iterations = len(sinogram)
    
    def draw_part_of_sinogram(sinogram, iteration):
        l  = [(line if i < iteration else [0] * len(line)) for i, line in enumerate(sinogram)]
        plt.figure(figsize = (10, 10))
        plt.imshow(l, cmap='binary_r')
        #return iteration

    interact(draw_part_of_sinogram, sinogram = fixed(sinogram), iteration = widgets.IntSlider(min=1, max=max_iterations, step=1, value=max_iterations))


def interactive_output(sinogram, original, out_size, phi):
    max_iterations = sinogram.shape[0]
    
    interact_manual(sinogram2img, sin = fixed(sinogram), original=fixed(original), picHeight = fixed(out_size[0]), picWidth = fixed(out_size[1]), phi = fixed(phi), iteration=widgets.IntSlider(min=1, max=max_iterations, step=1, value=max_iterations), filtered=False, show=fixed(True))

# Wczytywanie obrazu .jpg

In [ ]:
# Zmienne globalne
img = None
sinogram = None

In [ ]:
wName = widgets.Text(value='Kwadraty2.jpg', description='Nazwa pliku:', disabled=False)

wAlfaDelta = widgets.FloatSlider(value=2, min=0.25, max=2, step=0.25, description="Δα", continuous_update=False)

wN = widgets.IntSlider(value=180, min=90, max=720, step=90, description="L.detektorów", continuous_update=False)

wPhi = widgets.IntSlider(value=180, min=45, max=270, step=45, description="Rozpiętość", continuous_update=False)

wButtonSinogram = widgets.Button(description='Wygeneruj sinogram')

def gen_sinogram(_):
    global wName, wAlfaDelta, wN, wPhi
    global img, sinogram
    img = mpimg.imread('tomograf-zdjecia/'+wName.value)[:,:,2] / 255
    sinogram = img2sinogram(img, wAlfaDelta.value, wN.value, wPhi.value)
    
    picHeight = img.shape[0]
    picWidth = img.shape[1]
    interactive_sinogram(sinogram)
    interactive_output(sinogram, img, (picHeight, picWidth), wPhi.value)
    
wButtonSinogram.on_click(gen_sinogram)

display(wName, wAlfaDelta, wN, wPhi, wButtonSinogram)

# Zapis jako DICOM

In [ ]:
def save_dcm(image, filename, date, name, birth, sex, comment):
    file_meta = FileMetaDataset()
    file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.2'
    file_meta.MediaStorageSOPInstanceUID = "1.2.3"
    file_meta.ImplementationClassUID = "1.2.3.4"
    file_meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian

    ds = FileDataset(filename, {}, file_meta=file_meta, preamble=b"\0" * 128)
    ds.PatientName = name
    ds.PatientSex = sex
    ds.PatientBirthDate = birth.strftime('%Y%m%d')
    ds.PatientComments = comment

    ds.is_little_endian = True
    ds.is_implicit_VR = True
    ds.SeriesInstanceUID = pydicom.uid.generate_uid()
    ds.StudyInstanceUID = pydicom.uid.generate_uid()

    ds.ContentDate = date.strftime('%Y%m%d')
    ds.ContentTime = date.strftime('%H%M%S.%f')

    # 1 lub 3 - ile kolorów na obrazie
    ds.SamplesPerPixel = 1
    # interpretacja (MONOCHROME, RGB, HSV, ...)
    ds.PhotometricInterpretation = "MONOCHROME2"
    # 0 - unsigned int, 1 - U2
    ds.PixelRepresentation = 0
    # najwyższy bit
    ds.HighBit = 15
    # ilość bitów na piksel
    ds.BitsStored = 16
    # ilość bitów zaalokowanych na piksel
    ds.BitsAllocated = 16
    # minimalna wartość piksela - 0x0000
    ds.SmallestImagePixelValue = str.encode('\x00\x00')
    # maksymalna wartość piksela - 0xffff
    ds.LargestImagePixelValue = str.encode('\xff\xff')
    # kolumny, wiersze i dane
    ds.Columns = image.shape[1]
    ds.Rows = image.shape[0]
    ds.PixelData = (image * 65535).astype('uint16').tobytes()

    ds = correct_ambiguous_vr(ds, True)
    #print(ds)
    ds.save_as(filename, write_like_original=False)

In [ ]:
wDicomFilename = widgets.Text(value='Kwadraty2.dcm', description='Nazwa pliku')
wDicomPatientName = widgets.Text(value='Jan Kowalski', description='Imię i nazw.')
wDicomPatientBirth = widgets.Text(value='01.01.2000', description='Data ur.')
wDicomPatientBirthLabel = widgets.Label(value='Data powinna być zapisana w formacie DD.MM.YYYY')
wDicomDate = widgets.Text(value=datetime.datetime.now().strftime('%d.%m.%Y %H:%M'), description='Data badania')
wDicomDateLabel = widgets.Label(value='Data powinna być zapisana w formacie DD.MM.YYYY HH:MM')
wDicomPatientSex = widgets.RadioButtons(options=['M', 'F', 'O'], value='O', description='Płeć:')
wDicomComment = widgets.Textarea(placeholder='Wpisz komentarz...', description='Komentarz:')
wDicomFilter = widgets.Checkbox(value=False, description='Filtrowanie')

def wDicomSaveOnClick(_):
    global img, sinogram
    global wName, wAlfaDelta, wN, wPhi
    global wDicomFilename, wDicomDate, wDicomPatientName, wDicomPatientBirth, wDicomPatientSex
    global wDicomFilter, wDicomComment
    
    picHeight = img.shape[0]
    picWidth = img.shape[1]
    
    print("Generowanie obrazu wynikowego...")
    (result, error) = sinogram2img(sinogram,
                            picHeight,
                            picWidth,
                            wPhi.value,
                            sinogram.shape[0],
                            wDicomFilter.value)
    
    save_dcm(result,
             wDicomFilename.value,
             datetime.datetime.strptime(wDicomDate.value, '%d.%m.%Y %H:%M'),
             wDicomPatientName.value,
             datetime.datetime.strptime(wDicomPatientBirth.value, '%d.%m.%Y'),
             wDicomPatientSex.value,
             wDicomComment.value)
    print(f"Zapisano '{wDicomFilename.value}'")

wDicomSave = widgets.Button(description='Zapisz jako DICOM')
wDicomSave.on_click(wDicomSaveOnClick)

display(wDicomFilename, wDicomPatientName, wDicomPatientBirth, wDicomPatientBirthLabel, wDicomPatientSex)
display(wDicomDate, wDicomDateLabel, wDicomComment, wDicomFilter, wDicomSave)

# Odczyt DICOM

In [ ]:
def open_dcm(filename):
    ds = pydicom.dcmread(filename)
    
    print("Imię i nazwisko:", ds.PatientName)
    print("Data urodzenia:", datetime.datetime.strptime(ds.PatientBirthDate, '%Y%m%d').strftime('%d.%m.%Y') )
    print("Płeć:", ds.PatientSex)
    contentDate = datetime.datetime.strptime(ds.ContentDate, '%Y%m%d').strftime('%d.%m.%Y')
    contentTime = datetime.datetime.strptime(ds.ContentTime, '%H%M%S.%f').strftime('%H:%M')
    print("Data badania:", contentDate, contentTime)
    print("Komentarze:", ds.PatientComments)
    loaded = np.frombuffer(ds.PixelData, dtype='uint16').reshape(ds.Rows, ds.Columns)
    plt.figure(figsize = (10, 10))
    plt.imshow(loaded, cmap='binary_r')

In [ ]:
wDicomFilename2 = widgets.Text(value='Kwadraty2.dcm', description='Nazwa pliku')
wDicomOpen = widgets.Button(description='Otwórz DICOM')

def wDicomOpenOnClick(_):
    global wDicomFilename2
    open_dcm(wDicomFilename2.value)
wDicomOpen.on_click(wDicomOpenOnClick)

display(wDicomFilename2, wDicomOpen)

In [ ]:
# Eksperyment
img = mpimg.imread('tomograf-zdjecia/Shepp_logan.jpg')[:,:,2] / 255
height = img.shape[0]
width = img.shape[1]

alfaDelta = 2.0
n = 180
phi = 180

# for n in range(90, 720 + 1, 90):
#     sinogram = img2sinogram(img, alfaDelta, n, phi)
#     (result, error) = sinogram2img(sinogram, height, width, phi, sinogram.shape[0], filtered=True, show=False, original=img)
#     print(f"n = {n}, RMSE = {error}")
    
# for scans in range(90, 720 + 1, 90):
#     alfaDelta = 360.0 / scans
#     sinogram = img2sinogram(img, alfaDelta, n, phi)
#     (result, error) = sinogram2img(sinogram, height, width, phi, sinogram.shape[0], filtered=False, show=False, original=img)
#     print(f"skanów = {scans}, RMSE = {error}")
    
# for phi in range(45, 270 + 1, 45):
#     sinogram = img2sinogram(img, alfaDelta, n, phi)
#     (result, error) = sinogram2img(sinogram, height, width, phi, sinogram.shape[0], filtered=False, show=False, original=img)
#     print(f"phi = {phi}, RMSE = {error}")